In [1]:
!pip install transformers datasets evaluate scikit-learn gradio torch --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4
  At

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


In [3]:
dataset = load_dataset("ag_news")
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [4]:
print(dataset['train'].info)
print(dataset['test'].info)

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value('string'), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'])}, post_processed=None, supervised_keys=None, builder_name='parquet', dataset_name='ag_news', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=29832303, num_examples=120000, shard_lengths=None, dataset_name='ag_news'), 'test': SplitInfo(name='test', num_bytes=1880424, num_examples=7600, shard_lengths=None, dataset_name='ag_news')}, download_checksums={'hf://datasets/ag_news@eb185aade064a813bc0b7f42de02595523103ca4/data/train-00000-of-00001.parquet': {'num_bytes': 18585438, 'checksum': None}, 'hf://datasets/ag_news@eb185aade064a813bc0b7f42de02595523103ca4/data/test-00000-of-00001.parquet': {'num_bytes': 1234829, 'checksum': None}}, download_size=19820267, post_processing_size=None, dataset_size=31712727, size_in_bytes=51532994)
DatasetInfo(description='', citation='', homepage=''

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [15]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [16]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}


In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,  # try 8 if GPU RAM is low
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # just 1 epoch for testing
    report_to="none",    # disable wandb
    fp16=True,           # enable mixed precision
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-2685700641.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.187500,0.174632,0.945263,0.945249


TrainOutput(global_step=7500, training_loss=0.23579773661295572, metrics={'train_runtime': 1136.7872, 'train_samples_per_second': 105.561, 'train_steps_per_second': 6.598, 'total_flos': 3946736701440000.0, 'train_loss': 0.23579773661295572, 'epoch': 1.0})

In [21]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.17463183403015137, 'eval_accuracy': 0.9452631578947368, 'eval_f1': 0.9452490843393355, 'eval_runtime': 8.3956, 'eval_samples_per_second': 905.238, 'eval_steps_per_second': 56.577, 'epoch': 1.0}


In [22]:
trainer.save_model("./bert-ag-news")
tokenizer.save_pretrained("./bert-ag-news")


('./bert-ag-news/tokenizer_config.json',
 './bert-ag-news/special_tokens_map.json',
 './bert-ag-news/vocab.txt',
 './bert-ag-news/added_tokens.json',
 './bert-ag-news/tokenizer.json')

In [25]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [26]:
import gradio as gr

labels = ["World", "Sports", "Business", "Sci/Tech"]
def classify_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
    return {labels[i]: float(probs[0][i]) for i in range(len(labels))}


demo = gr.Interface(fn=classify_news, inputs="text", outputs="label")

demo.launch(share=True)  # share=True gives you a public Colab link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25b35d894fca23dd74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
print(classify_news("Apple stock price rises after iPhone 15 launch"))
print(classify_news("NASA announces new mission to explore Mars"))
print(classify_news("Lionel Messi scores a hat-trick for Inter Miami"))


{'World': 0.0032620218116790056, 'Sports': 0.00016577148926444352, 'Business': 0.14607279002666473, 'Sci/Tech': 0.8504994511604309}
{'World': 0.044751595705747604, 'Sports': 0.0002954130759462714, 'Business': 0.0009206623653881252, 'Sci/Tech': 0.9540322422981262}
{'World': 0.807172954082489, 'Sports': 0.18600332736968994, 'Business': 0.006073223892599344, 'Sci/Tech': 0.000750563689507544}


In [28]:
print(classify_news("Apple stock price rises after iPhone 15 launch"))


{'World': 0.0032620218116790056, 'Sports': 0.00016577148926444352, 'Business': 0.14607279002666473, 'Sci/Tech': 0.8504994511604309}
